In [3]:
#-*- coding: utf-8 -*-
from elasticsearch import Elasticsearch,JSONSerializer, helpers

import numpy as np
import pandas as pd
import json

# https://github.com/elastic/elasticsearch-py/issues/378
class NumpyEncoder(JSONSerializer):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return JSONSerializer.default(self, obj)
        
es = Elasticsearch(hosts="122.32.196.201:9200", port=9200, serializer=NumpyEncoder())  # 객체 생성

# Es Cluster Health Check
es.cluster.health() 


response = es.search(index='apt-trade', body={})
df = pd.DataFrame(response['hits']['hits'])
df = df['_source'].apply(pd.Series)

In [15]:
df

,transaction_amount,year_of_construction,transaction_year,legal_dong,apt_name,transaction_month,dedicated_area,floor,si,gu,sigungu,transaction_date,description,message
0,52500.0,1993,2016,분당동,샛별마을(라이프),4,84.99,18,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-01T00:00:00,경기도 성남시 분당구 분당동 샛별마을 라이프,샛별마을 라이프
1,46000.0,1993,2016,분당동,장안타운(건영),4,85.00,1,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-06T00:00:00,경기도 성남시 분당구 분당동 장안타운 건영,장안타운 건영
2,40500.0,1993,2016,분당동,장안타운(건영),4,70.68,7,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-06T00:00:00,경기도 성남시 분당구 분당동 장안타운 건영,장안타운 건영
3,42000.0,1992,2016,분당동,샛별마을(동성),4,69.39,4,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-07T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
4,35900.0,1992,2016,분당동,샛별마을(삼부),4,59.82,1,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-09T00:00:00,경기도 성남시 분당구 분당동 샛별마을 삼부,샛별마을 삼부
5,70000.0,1994,2016,분당동,샛별마을(우방),4,133.86,12,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-13T00:00:00,경기도 성남시 분당구 분당동 샛별마을 우방,샛별마을 우방
6,66500.0,1993,2016,분당동,샛별마을(라이프),4,126.42,16,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-13T00:00:00,경기도 성남시 분당구 분당동 샛별마을 라이프,샛별마을 라이프
7,39800.0,1994,2016,분당동,샛별마을(우방),4,57.28,10,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-16T00:00:00,경기도 성남시 분당구 분당동 샛별마을 우방,샛별마을 우방
8,56500.0,1992,2016,분당동,샛별마을(동성),4,101.88,2,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-17T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
9,38000.0,1992,2016,분당동,샛별마을(삼부),4,59.82,2,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-19T00:00:00,경기도 성남시 분당구 분당동 샛별마을 삼부,샛별마을 삼부


In [17]:
response = es.search(index='apt-trade', body={"query": {"match": {"apt_name": "샛별마을(동성)"}}})
df = pd.DataFrame(response['hits']['hits'])
df = df['_source'].apply(pd.Series)

In [18]:
df

,transaction_amount,year_of_construction,transaction_year,legal_dong,apt_name,transaction_month,dedicated_area,floor,si,gu,sigungu,transaction_date,description,message
0,42000.0,1992,2016,분당동,샛별마을(동성),4,69.39,4,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-07T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
1,56500.0,1992,2016,분당동,샛별마을(동성),4,101.88,2,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-17T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
2,50500.0,1992,2016,분당동,샛별마을(동성),4,84.82,10,경기도,성남시 분당구,경기도 성남시 분당구,2016-04-26T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
3,69000.0,1993,2016,분당동,샛별마을(동성),10,133.65,6,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-04T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
4,34500.0,1992,2016,분당동,샛별마을(동성),10,59.40,10,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-04T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
5,56000.0,1992,2016,분당동,샛별마을(동성),10,84.82,13,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-10T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
6,74000.0,1993,2016,분당동,샛별마을(동성),10,133.65,19,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-12T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
7,72000.0,1993,2016,분당동,샛별마을(동성),10,133.65,11,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-14T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
8,52000.0,1992,2016,분당동,샛별마을(동성),10,84.82,5,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-17T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성
9,52000.0,1992,2016,분당동,샛별마을(동성),10,84.82,5,경기도,성남시 분당구,경기도 성남시 분당구,2016-10-19T00:00:00,경기도 성남시 분당구 분당동 샛별마을 동성,샛별마을 동성


# 아파트 리스트 가져오기

In [5]:
response = es.search(index='apt-trade', body={'aggs': {'apts': {'terms': {'field': 'apt_name', 'size': 9999999}}}, 'size': 0})

In [17]:
df = pd.DataFrame(response['aggregations']['apts']['buckets'])
df = df.rename(columns={'key':'apt_name'})

In [18]:
df

,apt_name,doc_count
0,한솔마을(4단지)(주공),2402
1,시범한양,2277
2,장미마을(현대),1789
3,시범우성,1682
4,한솔마을(5단지)(주공),1588
...,...,...
185,산운마을9단지(대방노블랜드),16
186,산운마을11단지(주공)임대전환,15
187,산운마을12단지(주공)임대전환,6
188,서울시니어스분당타워,2
